# Weather Scraper

This notebook demonstrates how to scrape weather data from http://weather.com. We will look at the 5-day forecast, scrape the high temperatures, and then generate a graph to visualize them.

### First we read in the data and create the *soup*.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# it is good practice to identify yourself
# change the user-agent as appropriate
headers = {"user-agent": "WeatherBot/1.0"}
page = requests.get("https://weather.com/weather/5day/l/USCT0260:1:US", headers = headers)

if page.status_code != requests.codes.ok :
    print("Request was not successful, status code:", page.status_code)
    exit()
    
# Parse page using BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

### Scrape the data

In order to scrape the high temperature information we must understand the structure of the web page. The following observations are key:
- The table containing the forecast information has class = 'twc-table'. 
- Within each row of the table, the date is in a *span* element with class = 'date-time'
- Within each row of the table, high temp is in a *td* element with class = 'temp', which is stored in the first *span* element.

Let's first look at the first span tag for the first td element with class = 'temp':

In [ ]:
td = soup.find('td', {'class':'temp'})
td.span

Now extract the data, creating a list containing the days and a list containing the high temps.

In [ ]:
# find the table
table = soup.find('table', {'class':'twc-table'})
trs = table.find_all('tr')
days = []
temps = []
for rows in trs[1:] :
    date = rows.find('span', {'class':'date-time'})
    temp = rows.find('td', {'class':'temp'})
    theDate = date.string
    theHighTemp = temp.span.contents[0]
    print(theDate, ': ', theHighTemp, sep = '')
    if theHighTemp.isdigit() :
        days.append(theDate)
        temps.append(int(theHighTemp))

We will visualize the data by creating a bar plot with days on the $x$-axis and high temperatures on the $y$-axis. In order to do this, we first create a *pandas* data frame (table). The data frame is created by specifying a dictionary, where the keys will be the column names and the values will be the values for the corresponding column.

In [ ]:
df= pd.DataFrame(data = {"days":days, "hi_temp":temps})
df

Once we have a data frame, we can use the *plot.bar* method to create a bar plot from the data, where we specify the $x$ and $y$ values of the plot. We also set *rot* to 0 so that the bar labels are horizontal.

In [ ]:
plt = df.plot.bar(x = "days", y = "hi_temp", 
            title = "Willimantic High Temps, next 5 days",
            legend = False, rot = 0)

plt.set_xlabel("Day of the Week")
plt.set_ylabel("Temperature (Degrees Fahrenheit)")
None